Step 1: Load & initial audit

In [222]:
import pandas as pd
import numpy as np

# Step 1: Load & initial audit
file_path = "Airbnb_Open_Data.csv"

df = pd.read_csv(file_path)

df.head()

,id,host id,host_identity_verified,neighbourhood group,country,instant_bookable,cancellation_policy,room type,Construction year,price,service fee,minimum nights,number of reviews,reviews per month,review rate number,calculated host listings count,availability 365
0,1001254,80014485718,unconfirmed,Brooklyn,United States,False,strict,Private room,2020,966.0,193.0,10.0,9.0,0.210000,4,6.0,286.0
1,1002102,52335172823,verified,Manhattan,United States,False,moderate,Entire home/apt,2007,142.0,28.0,30.0,45.0,0.380000,4,2.0,228.0
2,1002403,78829239556,verified,Manhattan,United States,True,flexible,Private room,2005,620.0,124.0,3.0,0.0,1.375408,5,1.0,352.0
3,1002755,85098326012,unconfirmed,Brooklyn,United States,True,moderate,Entire home/apt,2005,368.0,74.0,30.0,270.0,4.640000,4,1.0,322.0
4,1003689,92037596077,verified,Manhattan,United States,False,moderate,Entire home/apt,2009,204.0,41.0,10.0,9.0,0.100000,3,1.0,289.0


In [223]:
attr_summary_step1 = pd.DataFrame({
    "dtype": df.dtypes.astype(str),
    "null_count": df.isna().sum(),
})

attr_summary_step1

,dtype,null_count
id,int64,0
host id,int64,0
host_identity_verified,object,0
neighbourhood group,object,0
country,object,0
instant_bookable,bool,0
cancellation_policy,object,0
room type,object,0
Construction year,int64,0
price,float64,0


Step 2: Duplicate Checking

In [224]:
# Step 2: Duplicate checking for listing id
id_dup_count = df["id"].duplicated().sum()
print(f"Duplicate id count: {id_dup_count}")

# Show sample duplicate ids if any
if id_dup_count > 0:
    dup_ids = df.loc[df["id"].duplicated(), "id"].head(10)
    print("Sample duplicate ids:", dup_ids.tolist())
else:
    print("No duplicate ids found.")


Duplicate id count: 0
No duplicate ids found.


In [225]:
# Remove duplicate rows by listing id
before_rows = len(df)
df = df.drop_duplicates(subset=["id"]).reset_index(drop=True)
removed = before_rows - len(df)
print(f"Removed {removed} duplicate rows. Remaining rows: {len(df)}")


Removed 0 duplicate rows. Remaining rows: 102058


Step 3: Missing value checking

In [226]:
# Step 3: Missing value report
missing_summary = pd.DataFrame({
    "missing_count": df.isna().sum(),
})

missing_summary

,missing_count
id,0
host id,0
host_identity_verified,0
neighbourhood group,0
country,0
instant_bookable,0
cancellation_policy,0
room type,0
Construction year,0
price,0


Step 4: Remove Outliner

In [227]:
# Clean 'minimum nights': set negative or >90 to NaN
min_nights_col = "minimum nights"
if min_nights_col in df.columns:
    df[min_nights_col] = pd.to_numeric(df[min_nights_col], errors="coerce")
    invalid_mask = (df[min_nights_col] < 0) | (df[min_nights_col] > 90)
    df.loc[invalid_mask, min_nights_col] = np.nan
    print(f"Set {invalid_mask.sum()} invalid '{min_nights_col}' values to NaN")
else:
    print(f"Column '{min_nights_col}' not found")


Set 0 invalid 'minimum nights' values to NaN


In [228]:
# Clean 'availability 365': set negative or >365 to NaN
avail_col = "availability 365"
if avail_col in df.columns:
    df[avail_col] = pd.to_numeric(df[avail_col], errors="coerce")
    invalid_mask = (df[avail_col] < 0) | (df[avail_col] > 365)
    df.loc[invalid_mask, avail_col] = np.nan
    print(f"Set {invalid_mask.sum()} invalid '{avail_col}' values to NaN")
else:
    print(f"Column '{avail_col}' not found")


Set 0 invalid 'availability 365' values to NaN


Step 5: Preparation for Filling Missing Value

In [229]:
# Reformat price and service fee to plain numeric
money_cols = ["price", "service fee"]
for col in money_cols:
    if col in df.columns:
        df[col] = (
            df[col]
            .astype(str)
            .str.replace("$", "", regex=False)
            .str.replace(",", "", regex=False)
        )
        df[col] = pd.to_numeric(df[col], errors="coerce")
        print(f"Reformatted '{col}' to numeric")
    else:
        print(f"Column '{col}' not found")

Reformatted 'price' to numeric
Reformatted 'service fee' to numeric


In [230]:
# Fix typos and standardize 'neighbourhood group'
if ng_col in df.columns:
    ng_series = df[ng_col].astype("string")
    ng_clean_base = ng_series.str.strip()
    ng_lower = ng_clean_base.str.lower()

    ng_map = {
        "manhatan": "Manhattan",
        "manhatten": "Manhattan",
        "manhattan": "Manhattan",
        "brooklyn": "Brooklyn",
        "brookln": "Brooklyn",
        "queens": "Queens",
        "bronx": "Bronx",
        "staten island": "Staten Island",
        "statenisland": "Staten Island",
    }

    ng_fixed = ng_lower.map(ng_map).fillna(ng_clean_base.str.title())
    changed_mask = ng_series.notna() & (ng_lower != ng_fixed.str.lower())

    if changed_mask.any():
        print(f"Fixed {int(changed_mask.sum())} '{ng_col}' values")
        print("Before -> After (counts):")
        change_summary = (
            pd.DataFrame({
                "before": ng_series[changed_mask].fillna("<NA>"),
                "after": ng_fixed[changed_mask].fillna("<NA>"),
            })
            .value_counts()
            .reset_index(name="count")
        )
        change_summary
    else:
        print(f"No typos detected in '{ng_col}'")

    df[ng_col] = ng_fixed
else:
    print(f"Column '{ng_col}' not found")

Fixed 1 'neighbourhood group' values
Before -> After (counts):


Step 6: Filling Missing Value

In [235]:
# Fill missing values
string_cols = [
    "host_identity_verified",
    "neighbourhood group",
    "country",
    "instant_bookable",
    "cancellation_policy",
]
float_max_cols = ["Construction year", "review rate number"]
float_mean_cols = [
    "price",
    "service fee",
    "number of reviews",
    "minimum nights",
    "reviews per month",
    "calculated host listings count",
    "availability 365",
]

for col in string_cols:
    if col in df.columns:
        fill_value = df[col].dropna().max()
        df[col] = df[col].fillna(fill_value)
        print(f"Filled missing '{col}' with max value: {fill_value}")
    else:
        print(f"Column '{col}' not found")

for col in float_max_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")
        fill_value = df[col].max()
        df[col] = df[col].fillna(fill_value)
        print(f"Filled missing '{col}' with max value: {fill_value}")
    else:
        print(f"Column '{col}' not found")

for col in float_mean_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")
        fill_value = df[col].mean()
        df[col] = df[col].fillna(fill_value)
        print(f"Filled missing '{col}' with mean value: {fill_value}")
    else:
        print(f"Column '{col}' not found")


Filled missing 'host_identity_verified' with max value: verified
Filled missing 'neighbourhood group' with max value: Staten Island
Filled missing 'country' with max value: United States
Filled missing 'instant_bookable' with max value: True
Filled missing 'cancellation_policy' with max value: strict
Filled missing 'Construction year' with max value: 2022
Filled missing 'review rate number' with max value: 5
Filled missing 'price' with mean value: 625.3555804381705
Filled missing 'service fee' with mean value: 125.03924939824608
Filled missing 'number of reviews' with mean value: 27.51794846626438
Filled missing 'minimum nights' with mean value: 7.112822438031374
Filled missing 'reviews per month' with mean value: 1.3754082791865605
Filled missing 'calculated host listings count' with mean value: 7.936936671285435
Filled missing 'availability 365' with mean value: 134.52848361636325


In [236]:
# Recheck missing values after cleaning
missing_summary_after = pd.DataFrame({
    "missing_count": df.isna().sum(),
})

missing_summary_after


,missing_count
id,0
host id,0
host_identity_verified,0
neighbourhood group,0
country,0
instant_bookable,0
cancellation_policy,0
room type,0
Construction year,0
price,0


Step 7: Remove Unused Column

In [233]:
# Drop unused columns and save cleaned dataframe
cols_to_drop = [
    "NAME",
    "host name",
    "neighbourhood",
    "lat",
    "long",
    "country code",
    "last review",
    "house_rules",
    "license",
]

cleaned_data = df.drop(columns=cols_to_drop, errors="ignore")
print(f"cleaned_data shape: {cleaned_data.shape}")


cleaned_data shape: (102058, 17)


In [234]:
# List attributes with data type and null count
attr_summary = pd.DataFrame({
    "dtype": cleaned_data.dtypes.astype(str),
    "null_count": cleaned_data.isna().sum(),
}).sort_index()

attr_summary

# Overwrite original CSV with cleaned data
cleaned_data.to_csv(file_path, index=False)
print(f"Saved cleaned_data to {file_path}")


Saved cleaned_data to Airbnb_Open_Data.csv


Step 9: Visualisation